In [1]:
import cv2
import pandas as pd
import numpy as np

from tqdm import tqdm


#import segmentation_models_pytorch as smp
import matplotlib.pyplot as plt
from IPython.display import clear_output 
import time
import os
import json
import random

random_seed = 42
np.random.seed(random_seed)
random.seed(random_seed)

In [2]:
# RLE 디코딩 함수
def rle_decode(mask_rle, shape):
    if mask_rle == -1:
        return np.zeros(shape, dtype=np.uint8)
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)

# RLE 인코딩 함수
def rle_encode(mask):
    if np.sum(mask) == 0:
        return '-1'
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [4]:
csv_files = []
csv_files.append(pd.read_csv('./result/b1_040_yjypost.csv'))  # CSV 파일 경로
csv_files.append(pd.read_csv('./result/b3_040_yjypost.csv'))

csv_files.append(pd.read_csv('./result/b4_040_yjypost.csv'))


csv_files.append(pd.read_csv('./result/b5_ade_yjypost.csv'))
csv_files.append(pd.read_csv('./result/b5_cityscapes_yjypost.csv')) #b5 48btch 42000steps
csv_files.append(pd.read_csv('./result/b5_vanila_yjypost.csv')) #b5 48btch 36000steps
### 재현불가능한 csv파일 ###

#csv_files.append(pd.read_csv('../submit/before42seed/b5_040_pre_yjypost.csv'))
#csv_files.append(pd.read_csv('../submit/before42seed/nvidiamit_b5_040_yjypost.csv'))
#csv_files.append(pd.read_csv('../submit/rubbish/b5_040_preprocessed_yjypost.csv'))

###



# 결과를 저장할 새로운 DataFrame 생성
result_data = pd.DataFrame(columns=['img_id', 'mask_rle'])

hflencsv=len(csv_files) // 2

for i in tqdm(range(9)):
    sumask=np.zeros((224,224), dtype=np.uint8)
    for csv_file in csv_files:
        mask_data = csv_file['mask_rle'][i]
        mask = rle_decode(mask_data, (224,224))
        sumask += mask
    sumask=np.where(sumask > hflencsv,1,0)

    result_data.loc[i] = [csv_file['img_id'][i], rle_encode(sumask)]
print(result_data.head)


100%|██████████| 9/9 [00:00<00:00, 208.58it/s]

<bound method NDFrame.head of   img_id                                           mask_rle
0      0  20103 4 20154 3 20326 8 20376 6 20550 7 20598 ...
1      1  35393 1 35617 2 35841 3 36065 4 36289 5 36513 ...
2      2                                                 -1
3      3  33564 7 33787 8 34009 10 34231 12 34451 16 346...
4      4  25108 9 25328 14 25551 16 25775 16 25999 16 26...
5      5  10753 3 10977 3 11201 4 11425 4 11649 5 11873 ...
6      6                                                 -1
7      7                                                 -1
8      8                                                 -1>


In [5]:
result_data.to_csv('./result/ensemble/ensemble0.csv', index=False)

In [6]:
print(len(csv_files))

6
